# GenAI-Based Resume Intelligence & Job Matching Platform

## Problem Statement
Recruiters spend significant time manually screening resumes to find suitable candidates for a job role.  
This project builds a GenAI-powered system that parses resumes, matches them with job descriptions using semantic similarity, and ranks candidates automatically.

## Objective
To automate resume screening and improve recruiter efficiency using LLMs and embedding-based similarity.

### Import the libraries

In [47]:
# Core libraries
import os
from pathlib import Path

# Resume & JD processing
import docx
import pandas as pd
import numpy as np

# Embeddings & similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import faiss

# Utilities
from tqdm import tqdm

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [48]:
# Define data paths

BASE_DIR = Path('..')

RESUME_DIR = BASE_DIR/'data'/'resumes'
JD_DIR = BASE_DIR/'data'/'job descriptions'/'job_title_des.csv'

print(f'Resume folder:',{RESUME_DIR})
print(f'JD folder:',{JD_DIR})

Resume folder: {WindowsPath('../data/resumes')}
JD folder: {WindowsPath('../data/job descriptions/job_title_des.csv')}


### Load Resumes and Extract the data

In [49]:
resume_data = []

for file in RESUME_DIR.glob("*.docx"):
    doc = docx.Document(file)
    text = " ".join([p.text for p in doc.paragraphs if p.text.strip()])
    
    resume_data.append({
        "file_name": file.name,
        "text": text
    })

In [50]:
resumes_df = pd.DataFrame(resume_data)

print(f"Total resumes loaded: {len(resumes_df)}")
resumes_df.head()

Total resumes loaded: 228


,file_name,text
0,Abiral_Pandey_Fullstack_Java.docx,Name: Abiral Pandey Email: abiral.pandey88@gma...
1,Achyuth Resume_8.docx,Achyuth 540-999-8048 achyuth.java88@gmail.com ...
2,Adelina_Erimia_PMP1.docx,"Adelina Erimia, PMP, Six Sigma Green Belt, SMC..."
3,Adhi Gopalam - SM.docx,Adhi Gopalam adhigopalam@gmail.com 281-212-359...
4,AjayKumar.docx,Ajay Kumar (CSM)\t \t\t Email/Skype: a...


This script loads multiple resume files in .docx format from a specified directory and prepares them for analysis. It iterates through each Word document, extracts non-empty paragraph text, and combines it into a single cleaned string representing the full resume content. For every resume, the file name and extracted text are stored as a dictionary and appended to a list. This list is then converted into a Pandas DataFrame, where each row corresponds to one resume and includes the resume file name and its textual content. Finally, the total number of resumes loaded is printed, and a preview of the dataset is displayed to verify successful extraction.

In [51]:
# Length of resumes
resumes_df["text_length"] = resumes_df["text"].apply(len)
resumes_df.head()

,file_name,text,text_length
0,Abiral_Pandey_Fullstack_Java.docx,Name: Abiral Pandey Email: abiral.pandey88@gma...,14185
1,Achyuth Resume_8.docx,Achyuth 540-999-8048 achyuth.java88@gmail.com ...,19290
2,Adelina_Erimia_PMP1.docx,"Adelina Erimia, PMP, Six Sigma Green Belt, SMC...",6088
3,Adhi Gopalam - SM.docx,Adhi Gopalam adhigopalam@gmail.com 281-212-359...,14170
4,AjayKumar.docx,Ajay Kumar (CSM)\t \t\t Email/Skype: a...,33364


In [52]:
role_keywords = {
    "Data Analyst": [
        "machine learning", "deep learning", "statistics", "model", "feature engineering",
        "pytorch", "tensorflow", "keras", "xgboost"
    ],
    "Data Engineer": [
        "spark", "hadoop", "kafka", "etl", "data pipeline", "airflow",
        "hive", "big data", "snowflake"
    ],
    "Software Engineer": [
        "java", "spring", "react", "javascript", "angular",
        "microservices", "api", "full stack"
    ],
    "QA / Testing": [
        "selenium", "test case", "qa", "automation testing", "junit", "manual testing"
    ],
    "PM / Scrum": [
        "scrum master", "product owner", "agile coach", "jira",
        "stakeholder", "program manager"
    ]
}

In [53]:
def classify_resume(text):
    text = text.lower()
    scores = {}

    for role, keywords in role_keywords.items():
        scores[role] = sum(1 for kw in keywords if kw in text)

    # pick role with highest keyword match
    best_role = max(scores, key=scores.get)

    if scores[best_role] == 0:
        return "Other"
    return best_role

In [54]:
resumes_df["predicted_role"] = resumes_df["text"].apply(classify_resume)

In [55]:
resumes_df["predicted_role"].value_counts()

predicted_role
Software Engineer    121
PM / Scrum            58
QA / Testing          24
Data Engineer         14
Data Analyst           7
Other                  4
Name: count, dtype: int64

In [56]:
resumes_df.head()

,file_name,text,text_length,predicted_role
0,Abiral_Pandey_Fullstack_Java.docx,Name: Abiral Pandey Email: abiral.pandey88@gma...,14185,Software Engineer
1,Achyuth Resume_8.docx,Achyuth 540-999-8048 achyuth.java88@gmail.com ...,19290,Software Engineer
2,Adelina_Erimia_PMP1.docx,"Adelina Erimia, PMP, Six Sigma Green Belt, SMC...",6088,PM / Scrum
3,Adhi Gopalam - SM.docx,Adhi Gopalam adhigopalam@gmail.com 281-212-359...,14170,PM / Scrum
4,AjayKumar.docx,Ajay Kumar (CSM)\t \t\t Email/Skype: a...,33364,QA / Testing


In [57]:
resumes_df[resumes_df['predicted_role'] == 'Data Analyst']

,file_name,text,text_length,predicted_role
22,BA - Abhishek.docx,Professional Summary Business Analyst skilled ...,20219,Data Analyst
30,Bharatha BA Resume.docx,PROFESSIONAL SUMMARY: 7 years of experience as...,10961,Data Analyst
38,Deepika Chintalapati.docx,Deepika\t\t\t\t\t\t\t 518-805-9569\nchintalapa...,17918,Data Analyst
39,Deepika DC.docx,Deepika\t\t\t\t\t\t\t 518-805-9569\nchintalapa...,16043,Data Analyst
76,Madhuri Pawar.docx,Madhuri Pawar 484 754 6872 madurichand@gmail.c...,15274,Data Analyst
200,Tejaswi-resume.docx,TEJASWI S\t\t\t\t\t\t\t\tEmployer Details sute...,10177,Data Analyst
201,TeresaNeetipudi IT BA.DOCX,TERESA NEETIPUDI teresaneet.ba@gmail.com (773)...,12177,Data Analyst


In [58]:
print(resumes_df.loc[200, 'text'][:500])

TEJASWI S								Employer Details sutejaswis@gmail.com					        Mail: Praveen@indiquesolutions.com        					                                      Phone: 703-743-0795 PROFESSIONAL PROFILE Senior Business/System Analyst with a strong technical background, energized by problem identification and resolution.  SUMMARY Over 7 years of experience in Business and Systems Analysis working in various domains – Financial, Mortgage, Retail and Healthcare.  Experienced in all phases of Software Develop


In [59]:
resumes_df['text_length'].describe()

count      228.000000
mean     18243.978070
std       7495.972814
min          0.000000
25%      13503.000000
50%      17766.000000
75%      22805.750000
max      51593.000000
Name: text_length, dtype: float64

In [60]:
resumes_df[resumes_df['text_length'] == resumes_df['text_length'].min()]

,file_name,text,text_length,predicted_role
43,employer_mounika details.docx,,0,Other
131,Raju Goduguchinta_Technical Program Manager.docx,,0,Other
150,"Resume Vishal PM - MSIS, PMP-PMI.docx",,0,Other
172,SAURABH_PM.docx,,0,Other


In [61]:
# drop resumes with 'zero-text'
resumes_df = resumes_df[resumes_df['text_length']>0]

In [62]:
resumes_df['text_length'].describe()

count      224.000000
mean     18569.763393
std       7149.867724
min       2794.000000
25%      13687.250000
50%      17853.500000
75%      23056.250000
max      51593.000000
Name: text_length, dtype: float64

In [63]:
print(resumes_df.loc[0, "text"][:1500])

Name: Abiral Pandey Email: abiral.pandey88@gmail.com Phone: 940-242-3303 Current Location: Woonsocket, Rhode Island Visa Status: US Citizen SUMMARY: Dynamic individual with 6 years of software development experience in design, development, deployment, maintenance, production and support of web - based and Client-Server business applications using OOP and Java/J2EE technologies. Exposure to all phases of Software Development Life Cycle(SDLC) using Agile, RUP, Waterfall. Designed and developed web UI screen using Angular-JS. Developed AngularJS Controllers, Services, filters and directives for various modules in the application. Knowledge on ETL tools like Kettle Pentaho and Microsoft SSIS tools. Created custom directives, decorators and services using AngularJS to interface with both RESTful and legacy network services also DOM applications. Experience with MVC frameworks like Struts, SPRING and ORM tools like Hibernate. Experienced in working with batch jobs using Spring-Batch, Autosys

### Load Job Descriptions and Explore the data

In [64]:
# Load Job Descriptions CSV
jd_df = pd.read_csv(JD_DIR)

print(f"Total job descriptions loaded: {len(jd_df)}")
jd_df.head()

Total job descriptions loaded: 2277


,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [65]:
jd_df.shape

(2277, 3)

In [66]:
jd_df['Job Title'].unique()

array(['Flutter Developer', 'Django Developer', 'Machine Learning',
       'iOS Developer', 'Full Stack Developer', 'Java Developer',
       'JavaScript Developer', 'DevOps Engineer', 'Software Engineer',
       'Database Administrator', 'Wordpress Developer', 'PHP Developer',
       'Backend Developer', 'Network Administrator', 'Node js developer'],
      dtype=object)

In [67]:
jd_df['text_length'] = jd_df['Job Description'].apply(len)

In [68]:
jd_df.head()

,Unnamed: 0,Job Title,Job Description,text_length
0,0,Flutter Developer,We are looking for hire experts flutter develo...,429
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...,503
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n...",1723
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...,1354
4,4,Full Stack Developer,job responsibility full stack engineer – react...,3261


In [69]:
print(jd_df.loc[0, "Job Description"][:1500])

We are looking for hire experts flutter developer. So you are eligible this post then apply your resume.
Job Types: Full-time, Part-time
Salary: ₹20,000.00 - ₹40,000.00 per month
Benefits:
Flexible schedule
Food allowance
Schedule:
Day shift
Supplemental Pay:
Joining bonus
Overtime pay
Experience:
total work: 1 year (Preferred)
Housing rent subsidy:
Yes
Industry:
Software Development
Work Remotely:
Temporarily due to COVID-19


In [70]:
jd_df['text_length'].describe()

count     2277.000000
mean      1986.595520
std       1442.771512
min        116.000000
25%        921.000000
50%       1604.000000
75%       2691.000000
max      10802.000000
Name: text_length, dtype: float64

### Load Embedding Model

In [71]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded")

✅ Embedding model loaded


In [72]:
# create resume embeddings
resume_text_list = resumes_df["text"].tolist()

resume_embeddings = embedding_model.encode(
    resume_text_list,
    show_progress_bar=True
)

print("Resume embeddings shape:", resume_embeddings.shape)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Resume embeddings shape: (224, 384)


In [73]:
# create Job description embedding
jd_text_lists = jd_df['Job Description'].tolist()

jd_embeddings = embedding_model.encode(
    jd_text_lists,
    show_progress_bar = True
)
print("JD embeddings shape:", jd_embeddings.shape)

Batches:   0%|          | 0/72 [00:00<?, ?it/s]

JD embeddings shape: (2277, 384)


### Similarity Matching & Ranking

In [74]:
# Select a job description index (you can change this)
jd_index = 4

selected_job_title = jd_df.iloc[jd_index]["Job Title"]
selected_jd_embedding = jd_embeddings[jd_index]

print("Selected Job Title:")
print(selected_job_title)

Selected Job Title:
Full Stack Developer


In [75]:
# Compute cosine similarity
similarity_scores = cosine_similarity(
    [selected_jd_embedding],   # shape: (1, 384)
    resume_embeddings          # shape: (num_resumes, 384)
)[0]  # flatten to 1D array

In [76]:
resumes_df['similarity_scores'] = similarity_scores

In [77]:
ranked_resumes = resumes_df.sort_values(
    by="similarity_scores",
    ascending=False
)

ranked_resumes.head(5)

,file_name,text,text_length,predicted_role,similarity_scores
175,Sharath Java.docx,Sharath Java Full Stack Developer sharath1323@...,13574,Software Engineer,0.537402
161,Sai Srinivas_Sr_Java_Developer.docx,Sai Srinivas Sr. Java/J2EE Developer ...,21867,Software Engineer,0.535856
188,srinivas b.docx,Srinivas Srinivasjava04@gmail.com 315-221-3723...,20394,Software Engineer,0.529652
210,Varun.docx,Varun OBJECTIVE: \t Seeking a position at you...,20276,Software Engineer,0.528939
11,Amulya Komatineni.docx,AMULYA KOMATINENI (515)309-1612 amulya.javadev...,26442,Software Engineer,0.521418


In [78]:
# Evaluate and validate
print(jd_df.loc[4, "Job Description"][:200])

job responsibility full stack engineer – react role make impact petsmart transforming engineering team meet need rapidly changing retail environment role foundational helping build craft petsmart ’ pr


In [79]:
print(resumes_df.loc[175, "text"][:200])

Sharath Java Full Stack Developer sharath1323@gmail.com +1-2245728262 SUMMARY: Java Full Stack developer with 7 years of experience in full software development life cycle including requirement analys


1) Role Alignment : Strong alignment with a Full Stack Engineer – React role, with extensive hands-on experience in ReactJS, SPA development, and full-stack Java-based web applications built in Agile environments.

1) Skill Overlap: ReactJS, JavaScript, HTML5, CSS3, responsive design, RESTful APIs, Java, Spring Boot, microservices, CI/CD, cloud deployment, and cross-functional collaboration.

1) Missing Skills: Limited explicit emphasis on advanced accessibility standards, large-scale React Native production deployments, and deep focus on frontend performance profiling at enterprise scale.

### Setting Up LLM

For Top N candidates, LLM will:
- Read Job Description, Read Resume, Read structured hints (optional)
Generate:
- Why the candidate matches, Strengths, Missing skills

In [80]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",   # cost-effective & good
    temperature=0.2        # low hallucination
)

In [82]:
# Create the prompt

explanation_prompt = PromptTemplate(
    input_variables=["job_description", "resume_text"],
    template="""
You are an AI assistant helping recruiters understand why a candidate matches a job.

Job Description:
{job_description}

Candidate Resume:
{resume_text}

Task:
- Explain why this candidate matches the job
- List key matching skills
- Mention any noticeable skill gaps
- Do NOT invent skills
- Keep the explanation concise and professional
"""
)

In [83]:
def generate_llm_explanation(job_desc, resume_text):
    prompt = explanation_prompt.format(
        job_description=job_desc,
        resume_text=resume_text
    )
    response = llm.invoke(prompt)
    return response.content

In [84]:
ranked_resumes.head()

,file_name,text,text_length,predicted_role,similarity_scores
175,Sharath Java.docx,Sharath Java Full Stack Developer sharath1323@...,13574,Software Engineer,0.537402
161,Sai Srinivas_Sr_Java_Developer.docx,Sai Srinivas Sr. Java/J2EE Developer ...,21867,Software Engineer,0.535856
188,srinivas b.docx,Srinivas Srinivasjava04@gmail.com 315-221-3723...,20394,Software Engineer,0.529652
210,Varun.docx,Varun OBJECTIVE: \t Seeking a position at you...,20276,Software Engineer,0.528939
11,Amulya Komatineni.docx,AMULYA KOMATINENI (515)309-1612 amulya.javadev...,26442,Software Engineer,0.521418


In [85]:
top_n = 3  # keep small to control cost

job_description_text = jd_df.iloc[jd_index]["Job Description"]

for i, row in ranked_resumes.head(top_n).iterrows():
    print(f"\n📄 Resume: {row['file_name']}")
    print(f"Match Score: {row['similarity_scores']:.4f}\n")

    explanation = generate_llm_explanation(
        job_description_text,
        row["text"][:]   # limit tokens
    )

    print("🧠 LLM Explanation:")
    print(explanation)
    print("=" * 80)


📄 Resume: Sharath Java.docx
Match Score: 0.5374

🧠 LLM Explanation:
### Candidate Match Explanation

The candidate, Sharath, is a strong match for the Full Stack Engineer role at PetSmart due to his extensive experience in full software development life cycles, particularly with web and mobile applications. He has over 7 years of experience in web development, including 2+ years of hands-on experience with React, which is a key requirement for this position. His background in Agile methodologies aligns well with the job's emphasis on delivering in an agile environment. Additionally, Sharath's experience with various JavaScript libraries and frameworks, including React Native, positions him well to contribute to building mobile-first customer experiences.

### Key Matching Skills
- **React and React Native**: 2+ years of recent experience with React and knowledge of React Native concepts.
- **Full Stack Development**: Proficient in both front-end (HTML5, CSS3, JavaScript, AngularJS) an

### FAISS Setup

In [91]:
# Normalize Resume Embeddings

import faiss
import numpy as np

# Normalize resume embeddings (important for cosine similarity)
resume_embeddings_np = np.array(resume_embeddings).astype("float32")
faiss.normalize_L2(resume_embeddings_np)

In [97]:
resume_embeddings_np

array([[-0.14330077,  0.01786945, -0.08012155, ..., -0.02880285,
        -0.00125058,  0.00242964],
       [-0.14571363,  0.04013793, -0.06846686, ..., -0.029992  ,
         0.00664964, -0.02061212],
       [-0.06761194, -0.01411534, -0.01394483, ..., -0.11545815,
        -0.11599265,  0.00315901],
       ...,
       [-0.03937842,  0.0318407 , -0.09887962, ..., -0.02196181,
        -0.03359338,  0.05969956],
       [-0.05740366,  0.06464995, -0.04114267, ..., -0.04312933,
         0.0433203 ,  0.01533818],
       [-0.10568763,  0.04288036, -0.0309347 , ..., -0.02214002,
         0.02631187,  0.04302999]], shape=(224, 384), dtype=float32)

In [101]:
# Create FAISS Index
embedding_dim = resume_embeddings_np.shape[1]

index = faiss.IndexFlatIP(embedding_dim)  # IP = inner product (cosine)
index.add(resume_embeddings_np)

print(f"✅ FAISS index created with {index.ntotal} resumes")

✅ FAISS index created with 224 resumes


In [103]:
# Normalize JD embedding
jd_embedding_np = np.array([selected_jd_embedding]).astype("float32")
faiss.normalize_L2(jd_embedding_np)

# Search top K resumes
top_k = 5
scores, indices = index.search(jd_embedding_np, top_k)

In [105]:
scores, indices

(array([[0.53740156, 0.535856  , 0.52965164, 0.5289389 , 0.5214181 ]],
       dtype=float32),
 array([[171, 158, 184, 206,  11]]))

In [104]:
print(f"\nTop {top_k} candidates using FAISS:\n")

for rank, (idx, score) in enumerate(zip(indices[0], scores[0]), start=1):
    resume = resumes_df.iloc[idx]
    print(f"Rank {rank}")
    print(f"Resume: {resume['file_name']}")
    print(f"FAISS Similarity Score: {score:.4f}")
    print("-" * 60)


Top 5 candidates using FAISS:

Rank 1
Resume: Sharath Java.docx
FAISS Similarity Score: 0.5374
------------------------------------------------------------
Rank 2
Resume: Sai Srinivas_Sr_Java_Developer.docx
FAISS Similarity Score: 0.5359
------------------------------------------------------------
Rank 3
Resume: srinivas b.docx
FAISS Similarity Score: 0.5297
------------------------------------------------------------
Rank 4
Resume: Varun.docx
FAISS Similarity Score: 0.5289
------------------------------------------------------------
Rank 5
Resume: Amulya Komatineni.docx
FAISS Similarity Score: 0.5214
------------------------------------------------------------


### SAVE & LOAD FAISS Index

In [106]:
# save faiss index to disk

import faiss
import os

# Create a folder to store FAISS index
INDEX_DIR = "faiss_index"
os.makedirs(INDEX_DIR, exist_ok=True)

# Path where index will be saved
FAISS_INDEX_PATH = os.path.join(INDEX_DIR, "resume_index.faiss")

# Save FAISS index to disk
faiss.write_index(index, FAISS_INDEX_PATH)

print("✅ FAISS index saved to disk")

✅ FAISS index saved to disk


In [107]:
load_index = faiss.read_index(FAISS_INDEX_PATH)

In [108]:
# Load FAISS index from disk
loaded_index = faiss.read_index(FAISS_INDEX_PATH)

print(f"✅ FAISS index loaded with {loaded_index.ntotal} vectors")

✅ FAISS index loaded with 224 vectors


In [109]:
# use loaded index for search
# Normalize JD embedding (same as before)
jd_embedding_np = np.array([selected_jd_embedding]).astype("float32")
faiss.normalize_L2(jd_embedding_np)

# Search using loaded FAISS index
top_k = 5
scores, indices = loaded_index.search(jd_embedding_np, top_k)

print("🔍 Search completed using loaded FAISS index")

🔍 Search completed using loaded FAISS index


## PHASE 2: CLEAN & MODULAR CODE

### 1) Resume Loading Module

In [117]:
from pathlib import Path
import docx
import pandas as pd

def load_resumes(resume_dir: Path) -> pd.DataFrame:
    resume_data = []

    for file in resume_dir.glob("*.docx"):
        doc = docx.Document(file)
        text = " ".join([p.text for p in doc.paragraphs if p.text.strip()])
        if text.strip():
            resume_data.append({
                "file_name": file.name,
                "text": text
            })

    return pd.DataFrame(resume_data)

### 2) JD Loading Module

In [112]:
def load_job_descriptions(jd_path: Path) -> pd.DataFrame:
    jd_df = pd.read_csv(jd_path)
    jd_df["Job Description"] = jd_df["Job Description"].astype(str).str.strip()
    return jd_df

### 3) Embedding Generation Module

In [113]:
from sentence_transformers import SentenceTransformer
import numpy as np

def generate_embeddings(texts: list[str], model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts, show_progress_bar=True)
    return np.array(embeddings).astype("float32")

### 4) FAISS Index Module

In [114]:
import faiss

def build_faiss_index(embeddings: np.ndarray):
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

### 5) FAISS Search Module

In [115]:
def search_candidates(index, query_embedding, top_k=5):
    query = np.array([query_embedding]).astype("float32")
    faiss.normalize_L2(query)
    scores, indices = index.search(query, top_k)
    return scores[0], indices[0]

### 6) LLM Explanation Module

In [118]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

explanation_prompt = PromptTemplate(
    input_variables=["job_description", "resume_text"],
    template="""
You are an AI assistant helping recruiters understand why a candidate matches a job.

Job Description:
{job_description}

Candidate Resume:
{resume_text}

Explain:
- Why the candidate matches
- Key matching skills
- Missing skills (if any)
Do NOT hallucinate.
"""
)

def generate_explanation(job_desc, resume_text):
    prompt = explanation_prompt.format(
        job_description=job_desc,
        resume_text=resume_text[:3000]
    )
    return llm.invoke(prompt).content

In [130]:
resumes_df.to_csv("resumes_metadata.csv", index=False)

In [132]:
import pandas as pd

df = pd.read_csv("resumes_metadata.csv")
df.head()

,file_name,text,text_length,predicted_role,similarity_scores
0,Abiral_Pandey_Fullstack_Java.docx,Name: Abiral Pandey Email: abiral.pandey88@gma...,14185,Software Engineer,0.418642
1,Achyuth Resume_8.docx,Achyuth 540-999-8048 achyuth.java88@gmail.com ...,19290,Software Engineer,0.387004
2,Adelina_Erimia_PMP1.docx,"Adelina Erimia, PMP, Six Sigma Green Belt, SMC...",6088,PM / Scrum,0.243179
3,Adhi Gopalam - SM.docx,Adhi Gopalam adhigopalam@gmail.com 281-212-359...,14170,PM / Scrum,0.222757
4,AjayKumar.docx,Ajay Kumar (CSM)\t \t\t Email/Skype: a...,33364,QA / Testing,0.374366


In [133]:
df.isna().sum()

file_name            0
text                 0
text_length          0
predicted_role       0
similarity_scores    0
dtype: int64